In [17]:
import pandas as pd
import numpy as np
from scipy import stats as sct
from statsmodels.stats import proportion 
import seaborn as sns
from matplotlib import pyplot as plt
import os

In [ ]:
df = pd.read_csv(os.path.join("dataset", "dataset_sp_limpo.csv"), index_col=0)
df = df[df.cs_sexo != "INDEFINIDO"]

## Test to verify if sexual gender influences COVID-19 mortality

H0: male mortality = female mortality

H1: male mortality ≠ female mortality

Alpha: 0.05

In [3]:
observed_frequency_death = df[df.obito == 1].cs_sexo.value_counts().values
observed_frequency_no_death = df[df.obito == 0].cs_sexo.value_counts().values

In [4]:
contingency_table = pd.DataFrame([observed_frequency_death, observed_frequency_no_death], 
                          index=["death", "no death"], columns=["male", "female"]).transpose()
contingency_table

,death,no death
male,20669,508373
female,15135,441954


In [5]:
stat, p_value, dof, expected = sct.chi2_contingency(contingency_table)
if p_value > 0.05:
    print("HO not reject")
else:
    print("H0 reject")

H0 reject


In [26]:
print("Male ratio death: {}".format(df[df.cs_sexo == "MASCULINO"].obito.mean()))
print("Female ratio death: {}".format(df[df.cs_sexo == "FEMININO"].obito.mean()))

Male ratio death: 0.04467784783722383
Female ratio death: 0.028910732978292594


Based on the conclusions that:
- The difference in mortality between men and women is significant
- Male mortality is higher than female 

We can come to another conclusion that men are more likely to die from COVID-19 than women